In [1]:
!pip install transformers[torch] datasets
!pip install --upgrade pyarrow
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
import os

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def preprocess_data(data, tokenizer):
    return data.map(lambda examples: tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512), batched=True)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!ls /content/drive/My\ Drive/dataset/ACCC/

ACCC.csv  ACCC_frames


In [13]:
def load_data(video_list, data_type):
    data = {'label': [], 'text': []}

    for video in video_list:
        video_data = pd.read_csv(f'/content/drive/My Drive/dataset/{video}/{video}.csv', header=None, skiprows=1, names=['label', 'text'])
        data['label'].extend(video_data['label'].tolist())
        data['text'].extend(video_data['text'].tolist())

    return data

In [14]:
test_videos = ['ACCFP', 'CCAH', 'CCSAD', 'CCUIM', 'EIB', 'EWCC', 'GGCC', 'SCCC', 'TICC', 'WICC']
val_videos = ['CCGFS', 'CCIAP', 'CICC', 'EFCC', 'FIJI', 'HCCAB', 'HRDCC', 'HUSNS', 'MACC', 'SAPFS']
train_videos = ['ACCC', 'AIAQ', 'AIDT', 'AMCC', 'BDCC', 'BECCC', 'BWFF', 'CBAQC', 'CCBN', 'CCBNN',
               'CCCBL', 'CCCP', 'CCCS', 'CCD', 'CCFS', 'CCFWW', 'CCH', 'CCHES', 'CCIAA', 'CCIAH', 'CCICD',
               'CCIS', 'CCISL', 'CCMA', 'CCSC', 'CCTA', 'CCTP', 'CCWC', 'CCWQ', 'CESS', 'COP',
               'CPCC', 'CTCM', 'DACC', 'DFCC', 'DPIC', 'DTECC', 'ECCDS', 'FCC', 'FLW', 'FTACC',
               'HCCAE', 'HCCAW', 'HCCIG', 'HCI', 'HDWC', 'HHVBD', 'HSHWA', 'HSPW', 'IMRF', 'INCAS',
               'MICC', 'NASA', 'OCCC', 'PCOCC', 'PWCCA', 'RAGG', 'RASCC', 'RCCCS', 'RCCS', 'RHTCC',
               'RPDCC', 'SDDA', 'SLCCA', 'SSTCC', 'TCBCC', 'TECCC', 'TIOCC', 'TIYH', 'TTFCC',
               'TUCC', 'UKCC', 'VFVCC', 'VPCC', 'WCCA', 'WFHSW', 'WICCE', 'WISE', 'WTCC', 'YPTL']

In [15]:
train_data = load_data(train_videos, 'train')
val_data = load_data(val_videos, 'val')
test_data = load_data(test_videos, 'test')

In [16]:
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

In [17]:
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [18]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3387 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

In [19]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
def compute_metrics(p):
    pred_labels = p.predictions.argmax(-1)
    accuracy = accuracy_score(p.label_ids, pred_labels)
    f1 = f1_score(p.label_ids, pred_labels, average='weighted')
    return {"accuracy": accuracy, "f1": f1}

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.725700,0.624780,0.787234,0.788090
2,0.449900,0.629932,0.799054,0.800594
3,0.278900,0.636536,0.794326,0.796104


TrainOutput(global_step=636, training_loss=0.5684048634655071, metrics={'train_runtime': 994.0193, 'train_samples_per_second': 10.222, 'train_steps_per_second': 0.64, 'total_flos': 2673495437534208.0, 'train_loss': 0.5684048634655071, 'epoch': 3.0})

In [23]:
trainer.evaluate(tokenized_datasets['test'])

{'eval_loss': 0.7165553569793701,
 'eval_accuracy': 0.7047619047619048,
 'eval_f1': 0.7045126399353804,
 'eval_runtime': 12.6995,
 'eval_samples_per_second': 33.072,
 'eval_steps_per_second': 2.126,
 'epoch': 3.0}